In [10]:
!pip install datasets torch transformers accelerate sentence_transformers matplotlib


In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define model name
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Load model with automatic device placement
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float32,  # Use float16 for efficiency
    device_map="auto",  # Automatically assigns to GPU if available
    trust_remote_code=True  # Required for some models
).to("cuda")

print(f"✅ Model and tokenizer loaded successfully on {model.device}")



✅ Model and tokenizer loaded successfully on cuda:0


In [12]:
# Load MATH-500 dataset
from datasets import load_dataset


print("📥 Loading MATH-500 dataset...")
dataset = load_dataset("HuggingFaceH4/MATH-500")

📥 Loading MATH-500 dataset...


# I was having problems parsing the correct answer from the Math response:
So I found this library https://github.com/huggingface/Math-Verify that does a pretty good job.
However I did add some redundancy where I collected 3 responses per question.

In [13]:
!pip install math-verify[antlr4_13_2]

# I ran the code below twice: 
- once for 100 problems (which wasn't enough data for the aggregated studies I was trying to do)
- second time for 200 problems

I noticed that sometimes the parser wouldn't find the correct solution even though the model would provide it, so in addition to comparing against the 'answer' field I also parse the 'solution' field so that I give the parser the best chance of finding the solution.

I basically want the best chance of finding a correct solution (if the model has provided it)

In [ ]:
import torch
import re
import numpy as np
from math_verify import parse, verify
from collections import defaultdict

def generate_math_answer(question):
    """
    Generates an answer for a given math problem.

    Parameters:
    - question (str): The math word problem.

    Returns:
    - answer (str): The generated answer response.
    """
    print(f"🤖 Generating answer for: {question[:50]}...")  # Debugging

    input_text = (
        f"""Solve the following math problem step by step. {question}. The last line of your response should be of the form Answer:
"""
    )

    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    model_output = model.generate(
    **inputs, 
    max_new_tokens=1024,  
    temperature=0.7,  
    top_p=1.0,  
    pad_token_id=tokenizer.eos_token_id
    )

    # If model_output is a tensor, index it directly
    generated_ids = model_output[0] if isinstance(model_output, torch.Tensor) else model_output.sequences[0]

    answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return answer

# Run and collect correctness responses
total_correctness = []
num_samples = 3  # Number of times to generate an answer per question

print("🚀 Generating answers for MATH dataset...")
model_answers = defaultdict()
for idx, item in enumerate(dataset["test"].select(range(100))):  
    print(f"\n📌 Processing sample {idx+1}/120")

    try:
        question = item["problem"]
        correct_answer = item["answer"]
        alternative_answer = item["solution"]
    except KeyError as e:
        print(f"❌ Missing expected field: {e}")
        continue

    correctness_results = []

            
    model_answers[idx] = defaultdict()
    model_answers[idx]['question'] = question
    
    for sample_idx in range(num_samples):
        model_answers[idx][sample_idx] = defaultdict()
        model_answer = generate_math_answer(question)

        
        # Extract gold answer from dataset
        gold_answer = parse(correct_answer)
        generated_answer = parse(model_answer)
        alternative_answer = parse(alternative_answer)
        print('Gold', gold_answer)
        print('Generated', generated_answer)
        print('Alternative', alternative_answer)
        
        # Verify correctness using math_verify
        is_correct = verify(gold_answer, generated_answer) or verify(alternative_answer, generated_answer)
        print('Correct', is_correct)
        
        correctness_results.append(int(is_correct))
        model_answers[idx][sample_idx] = {'raw_model_answer': model_answer, 'parsed_model_answer': generated_answer, 'parsed_correct_answer': gold_answer, 'parsed_solution': alternative_answer}

    # Calculate average correctness over the samples
    avg_correctness = np.mean(correctness_results)
    model_answers[idx]['correctness_list'] = correctness_results
    model_answers[idx]['average_correctness'] = avg_correctness

    # Store correctness values
    total_correctness.append(avg_correctness)

    # Print results for each sample
    print(f"\n📌 Sample {idx+1}")
    print(f"🔹 Question: {question}")
    print(f"🔹 Real Answer: {correct_answer}")
    print(f"🔹 Average Correctness Over {num_samples} Samples: {avg_correctness:.2f}")

# Compute and display overall average correctness
overall_correctness = np.mean(total_correctness)
print(f"\n📊 Overall Average Correctness Across {len(total_correctness)} Questions: {overall_correctness:.2f}")

print(model_answers)

🚀 Generating answers for MATH dataset...

📌 Processing sample 1/120
🤖 Generating answer for: Convert the point $(0,3)$ in rectangular coordinat...


Gold [3, '3']
Generated [0, '0']
Alternative [(3, pi/2), '( 3, \\frac{\\pi}{2} )']
Correct False
🤖 Generating answer for: Convert the point $(0,3)$ in rectangular coordinat...


Error during parsing
Traceback (most recent call last):
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 640, in parse
    return extract_target_from_pred(pred, target_res, fallback_mode=fallback_mode, extraction_mode=extraction_mode, timeout_seconds=parsing_timeout)
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 568, in extract_target_from_pred
    matches_with_pos = sorted(
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 564, in <genexpr>
    for match in pattern.finditer(pred)
TypeError: expected string or bytes-like object


Gold [3, '3']
Generated [0, '0']
Alternative []
Correct False
🤖 Generating answer for: Convert the point $(0,3)$ in rectangular coordinat...


Error during parsing
Traceback (most recent call last):
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 640, in parse
    return extract_target_from_pred(pred, target_res, fallback_mode=fallback_mode, extraction_mode=extraction_mode, timeout_seconds=parsing_timeout)
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 568, in extract_target_from_pred
    matches_with_pos = sorted(
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 564, in <genexpr>
    for match in pattern.finditer(pred)
TypeError: expected string or bytes-like object


...

Gold [2, '2']
Generated [2*k + 2, '2k + 2']
Alternative []
Correct False

📌 Sample 99
🔹 Question: Simplify $(-k + 4) + (-2 + 3k)$.
🔹 Real Answer: 2k+2
🔹 Average Correctness Over 3 Samples: 0.33

📌 Processing sample 100/120
🤖 Generating answer for: If $\mathbf{a} = \begin{pmatrix} 1 \\ 1 \\ 1 \end{...
Gold [5/3, '5/3']
Generated [5, '5/3).']
Alternative [Matrix([
[-1/3],
[ 2/3],
[ 5/3]]), '\\begin{pmatrix} -1/3 \\\\ 2/3 \\\\ 5/3 \\end{pmatrix}']
Correct False
🤖 Generating answer for: If $\mathbf{a} = \begin{pmatrix} 1 \\ 1 \\ 1 \end{...


Error during parsing
Traceback (most recent call last):
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 640, in parse
    return extract_target_from_pred(pred, target_res, fallback_mode=fallback_mode, extraction_mode=extraction_mode, timeout_seconds=parsing_timeout)
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 568, in extract_target_from_pred
    matches_with_pos = sorted(
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 564, in <genexpr>
    for match in pattern.finditer(pred)
TypeError: expected string or bytes-like object


Gold [5/3, '5/3']
Generated [{-1/3, 2/3, 5/3}, '-1/3, 2/3, 5/3']
Alternative []
Correct False
🤖 Generating answer for: If $\mathbf{a} = \begin{pmatrix} 1 \\ 1 \\ 1 \end{...


Error during parsing
Traceback (most recent call last):
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 640, in parse
    return extract_target_from_pred(pred, target_res, fallback_mode=fallback_mode, extraction_mode=extraction_mode, timeout_seconds=parsing_timeout)
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 568, in extract_target_from_pred
    matches_with_pos = sorted(
  File "/home/mosaicml/Development/cot/.venv/lib/python3.10/site-packages/math_verify/parser.py", line 564, in <genexpr>
    for match in pattern.finditer(pred)
TypeError: expected string or bytes-like object


Gold [5/3, '5/3']
Generated [Matrix([
[ 5/3],
[ 2/3],
[-1/3]]), '\\begin{pmatrix} \\frac{5}{3} \\\\ \\frac{2}{3} \\\\ -\\frac{1}{3} \\end{pmatrix}']
Alternative []
Correct False

📌 Sample 100
🔹 Question: If $\mathbf{a} = \begin{pmatrix} 1 \\ 1 \\ 1 \end{pmatrix},$ then find the vector $\mathbf{v}$ such that $\mathbf{a} \cdot \mathbf{v} = 2$ and $\mathbf{a} \times \mathbf{v} = \begin{pmatrix} 1 \\ -2 \\ 1 \end{pmatrix}.$
🔹 Real Answer: \begin{pmatrix} -1/3 \\ 2/3 \\ 5/3 \end{pmatrix}
🔹 Average Correctness Over 3 Samples: 0.00

📊 Overall Average Correctness Across 100 Questions: 0.43
defaultdict(None, {0: defaultdict(None, {'question': 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$', 0: {'raw_model_answer': "Solve the following math problem step by step. Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta

In [21]:
model_answers[1]

defaultdict(None,
            {'question': 'Define\n\\[p = \\sum_{k = 1}^\\infty \\frac{1}{k^2} \\quad \\text{and} \\quad q = \\sum_{k = 1}^\\infty \\frac{1}{k^3}.\\]Find a way to write\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3}\\]in terms of $p$ and $q.$',
             0: {'raw_model_answer': 'Solve the following math problem step by step. Define\n\\[p = \\sum_{k = 1}^\\infty \\frac{1}{k^2} \\quad \\text{and} \\quad q = \\sum_{k = 1}^\\infty \\frac{1}{k^3}.\\]Find a way to write\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3}\\]in terms of $p$ and $q.$. The last line of your response should be of the form Answer:\n</think>\n\nTo solve the problem, we start by expanding the given double sum:\n\n\\[\n\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3}\n\\]\n\nWe can rewrite this as:\n\n\\[\n\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3} = \\sum_{n = 2}^\\infty \\sum_{j = 1}^{n - 1} \\frac{1}{n^3}\n\\]\n\nwhere

# Save the object:
as a pickle file

In [52]:
import dill as pickle
import collections

def convert_safe(obj):
    """Recursively converts unpicklable objects to strings."""
    if isinstance(obj, collections.defaultdict):
        return {k: convert_safe(v) for k, v in obj.items()}  # Fixed syntax
    elif isinstance(obj, dict):
        return {k: convert_safe(v) for k, v in obj.items()}  # Fixed syntax
    elif isinstance(obj, list):
        return [convert_safe(i) for i in obj]
    elif isinstance(obj, tuple):
        return tuple(convert_safe(i) for i in obj)
    elif isinstance(obj, (int, float, str, bool, type(None))):
        return obj
    return str(obj)  # Convert unpicklable objects (e.g., `Equality`) to strings

# Convert model_answers before saving
safe_model_answers = convert_safe(model_answers)

# Save as Pickle
with open("model_answers_fixed.pkl", "wb") as f:
    pickle.dump(safe_model_answers, f)

print("✅ Successfully saved a pickle-safe version!")


✅ Successfully saved a pickle-safe version!
